<a href="https://colab.research.google.com/github/chloevan/gcp_tutorial/blob/master/02_basics/Ch10_Unnest_an_Array.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
title: "Ch10 UNNEST an Array"
date: 2020-05-02T19:30:30+09:00
tags:
  - "구글 클라우드"
  - "Google Cloud"
  - "빅쿼리"
  - "BigQuery"
  - "빅쿼리 파이썬"
  - "빅쿼리 기본"
  - "UNNEST"
  - "BigQuery with Python"
categories:
  - "구글 클라우드"
  - "Google Cloud"
  - "빅쿼리"
  - "BigQuery"
  - "Python"
menu: 
  gcp:
    name: Ch09_Array_Of_Struct
---

## 1. 구글 클라우드 설정

본격적인 빅쿼리 실습에 앞서서, Python과 연동하는 예제를 준비하였다. 빅쿼리 시작에 앞서서 선행적으로 클라우드 사용을 해야 한다. 

1. 만약 GCP 프로젝트가 없다면, 계정을 연동한다. Go to [Cloud Resource Manager](https://console.cloud.google.com/cloud-resource-manager)
2. 그리고, 비용결제를 위한 카드를 등록한다. [Enable billing](https://support.google.com/cloud/answer/6293499#enable-billing) 
3. 마지막으로 BigQuery API를 사용해야 하기 때문에 빅쿼리 API 사용허가를 내준다.[Enable BigQuery](https://console.cloud.google.com/flows/enableapi?apiid=bigquery)

위 API를 이용하지 않으면 `Python` 또는 `R`과 연동해서 사용할 수는 없다. 자주 쓰는것이 아니라면 비용은 거의 발생하지 않으니 염려하지 않아도 된다. 비용관리에 대한 자세한 내용은 [BigQuery 권장사항: 비용 관리](https://cloud.google.com/bigquery/docs/best-practices-costs?hl=ko)에서 확인하기를 바란다.  


## 2. 사용자 계정 인증
구글 코랩을 사용해서 인증 절차를 밟도록 한다. 아래 소스코드는 변경시키지 않는다. 아래 절차대로 진행하면 된다. `Gmail` 인증 절차와 비슷하다.

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## 3. 쿼리 기본
빅쿼리는 기본적으로 데이터 웨어하우스의 기능을 가지고 있다. Structured(데이터프레임) & Semi-Structured(JSON)과 같은 DB를 지원하고 있다. Create, Read, Update, Delete 기능을 지원한다. 

빅쿼리는 데이터 분석을 위한 기본적인 도구이기 때문에, 웹/앱을 활용한 마케팅 분석이 필요한 업종에서는 어쩌면 반드시 가져가야할 일종의 언어이기도 하다. 

기본적으로 SQL문법을 지원하며, BigQuery 나름의 문법을 또한 구성하고 있다. 이제 본격적으로 코드를 작성해보자. 

현재 작성하는 모든 코드는 Python-BigQuery로 연동하여 작성하려고 한다. 이제 간단하게 쿼리를 작성해보자. 

- [Ch05-Query_Essentials(1)](https://chloevan.github.io/gcp/bigquery/02_basics/ch05_query_essentials1/)에서는 `SELECT`와 `AS`에 대해 배우는 시간을 가졌다. 
- [Ch05-Query_Essentials(2)](https://chloevan.github.io/gcp/bigquery/02_basics/ch05_query_essentials2/)에서는 `WHERE`와 `EXCEPT`, 그리고 `REPLACE`에 대해 배우는 시간을 가졌다.
- [Ch05-Query_Essentials(3)](https://chloevan.github.io/gcp/bigquery/02_basics/ch05_query_essentials3/)에서는 `SubQuery`에 대해서 집중적으로 배우는 시간을 가졌다.  
- [Ch06 SQL Aggregates](https://chloevan.github.io/gcp/bigquery/02_basics/ch06_sql_aggregates/)에서는 집계함수를 사용해서 데이터를 조금 더 디테일하게 요약하는 것을 배웠다.
- [Ch07 Arrays & Structs](https://chloevan.github.io/gcp/bigquery/02_basics/ch07_split_unnest/)에서는 빅쿼리에서 가상의 테이블과 그리고 배열을 만드는 법을 익혔다. 
- [Ch08_Creating_Arrays_with_Array_AGG](https://chloevan.github.io/gcp/bigquery/02_basics/ch08_creating_arrays_with_array_agg/)
- [Ch09_Array_Of_Struct](https://chloevan.github.io/gcp/bigquery/02_basics/ch09_struct_and_tuple/) Struct을 활용하여 독특한 쿼리 결과문(`Array`)을 확인하였다. 


### (1) UNNEST

우선 지난 시간에 `Struct`을 통해서 Array 결과문을 확인해본다. 

In [3]:
from google.cloud import bigquery
from tabulate import tabulate

project_id = 'bigquerytutorial-274406'
client = bigquery.Client(project=project_id)

temp = client.query('''
  SELECT
    [
      STRUCT('male' as gender, [930660, 3955871] as numtrips)
      , STRUCT('female' as gender, [3236735, 1260893] as numtrips) 
    ]
  ''').to_dataframe()

print(tabulate(temp, tablefmt="pipe", headers="keys"))

|    | f0_                                                                                                       |
|---:|:----------------------------------------------------------------------------------------------------------|
|  0 | [{'gender': 'male', 'numtrips': [930660, 3955871]}, {'gender': 'female', 'numtrips': [3236735, 1260893]}] |


위 구문에서 확인할 수 있는 것처럼, 하나의 `Cell` 성별에 다른 `numtrips` 정보가 담긴 것을 확인할 수 있다. 많은 정보를 담을 수는 있지만, 그러나 가독성은 당연히 떨어진다. 

`gender`와 `numtrips`가 필드가 될 수 있도록 구성해보자. 

그러나, 이 쿼리가 종착역이 될 수는 없다. 즉, 우리에게 익숙한 `Table` or `JSON` 형태로 다시 만들어져야 한다. 

그런데, 위 구문을 `UNNEST`로 받으면 매우 쉽게 결과물이 나오게 된다. 

In [4]:
temp = client.query('''
  SELECT * FROM UNNEST(
    [
      STRUCT('male' as gender, [930660, 3955871] as numtrips)
      , STRUCT('female' as gender, [3236735, 1260893] as numtrips) 
    ]
  )
  ''').to_dataframe()

print(tabulate(temp, tablefmt="pipe", headers="keys"))

|    | gender   | numtrips           |
|---:|:---------|:-------------------|
|  0 | male     | [930660, 3955871]  |
|  1 | female   | [3236735, 1260893] |


그런데, 이 때, `SELECT * `에서 `*` 대신에 `Field`명을 입력하면 해당 필드만 조회되게 된다. 

다음과 같이 해보자. 

In [5]:
temp = client.query('''
  SELECT numtrips FROM UNNEST(
    [
      STRUCT('male' as gender, [930660, 3955871] as numtrips)
      , STRUCT('female' as gender, [3236735, 1260893] as numtrips) 
    ]
  )
  ''').to_dataframe()

print(tabulate(temp, tablefmt="pipe", headers="keys"))

|    | numtrips           |
|---:|:-------------------|
|  0 | [930660, 3955871]  |
|  1 | [3236735, 1260893] |


위와 같이 결과물이 나온 것을 확인할 수 있다. 이제, 실전에 들어간 준비는 끝냈다. 실전에서 어떻게 쓸 수 있을까?

다음 Chapter에서 보다 상세하게 작성을 하도록 하겠다. (Comming Soon) 

 

## 4. Reference
Lakshmanan, V., & Tigani, J. (2020). Google BigQuery: the definitive guide: data warehousing, analytics, and machine learning at scale. Beijing: OReilly.